<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


## Package Downloads

In [1]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [2]:
import folium
import pandas as pd

In [3]:
from folium.features import DivIcon


In [4]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

In [5]:
import os

# List all files in the current directory
print(os.listdir())

['Untitled.ipynb', 'lab_jupyter_launch_site_location.jupyterlite.ipynb', 'spacex_launch_geo.csv', 'untitled5.py']


## Task 1: Mark all launch sites on a map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


### Download and read the `spacex_launch_geo.csv`

In [6]:
import pandas as pd

# Load the dataset
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# Display the first few rows of the DataFrame to confirm it loaded correctly
print(spacex_df.head())

# Verify the total number of rows and the breakdown by launch site
print(f"Total Launches: {len(spacex_df)}")
print(spacex_df["Launch Site"].value_counts())

   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \
0        LEO           SpaceX  Failure   (

Now, you can take a look at what are the coordinates for each site.


### Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`

In [7]:
import pandas as pd

# Load the dataset
spacex_df = pd.read_csv("spacex_launch_geo.csv")

# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]

# Group by `Launch Site` and get the first occurrence for each site
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()

# Keep only `Launch Site`, `Lat`, and `Long` columns
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]

# Display the DataFrame
print(launch_sites_df)

    Launch Site        Lat        Long
0   CCAFS LC-40  28.562302  -80.577356
1  CCAFS SLC-40  28.563197  -80.576820
2    KSC LC-39A  28.573255  -80.646895
3   VAFB SLC-4E  34.632834 -120.610745


#### Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


#### We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


### Start location is NASA Johnson Space Center

In [8]:
# Start location is NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [9]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))

# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


### Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [10]:
import folium
from folium.features import DivIcon

# Coordinates for the launch sites
launch_sites = [
    {"name": "CCAFS LC-40", "lat": 28.562302, "lon": -80.577356},
    {"name": "CCAFS SLC-40", "lat": 28.563197, "lon": -80.576820},
    {"name": "KSC LC-39A", "lat": 28.573255, "lon": -80.646895},
    {"name": "VAFB SLC-4E", "lat": 34.632834, "lon": -120.610745},
]

# Initialize the map at a central point
site_map = folium.Map(location=[28.5, -80.6], zoom_start=5)

# Add a circle and a marker for each launch site
for site in launch_sites:
    # Add a Circle
    folium.Circle(
        location=[site["lat"], site["lon"]],
        radius=1000,  # Circle radius in meters
        color='#000000',  # Circle border color
        fill=True,
        fill_color='#3186cc'  # Circle fill color
    ).add_to(site_map)
    
    # Add a Marker with a custom text label
    folium.Marker(
        location=[site["lat"], site["lon"]],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12; color: #d35400;"><b>{site["name"]}</b></div>',
        )
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


## Task 2: Mark the success/failed launches for each site on the map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [11]:
import pandas as pd
from folium.plugins import MarkerCluster
import folium

# Step 1: Load the SpaceX data from the CSV file
try:
    spacex_df = pd.read_csv("spacex_launch_geo.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: The file 'spacex_launch_geo.csv' was not found. Make sure it is in the same directory as this notebook.")
    raise

# Step 2: Verify the total number of launches
total_launches = len(spacex_df)
print(f"Total Launches: {total_launches}")

# Step 3: Display the first few rows of the DataFrame
print(spacex_df.head())

Dataset loaded successfully.
Total Launches: 56
   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        L

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` valueimport pandas as pd
from folium.plugins import MarkerCluster

In [12]:
# Step 4: Add a new column 'marker_color' to the DataFrame based on the 'class' value
try:
    spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')
    print("Column 'marker_color' added successfully.")
except KeyError:
    print("Error: The 'class' column is missing from the dataset.")
    raise

# Step 5: Display the updated DataFrame
print(spacex_df.head())

Column 'marker_color' added successfully.
   Flight Number        Date Time (UTC) Booster Version  Launch Site  \
0              1  2010-06-04   18:45:00  F9 v1.0  B0003  CCAFS LC-40   
1              2  2010-12-08   15:43:00  F9 v1.0  B0004  CCAFS LC-40   
2              3  2012-05-22    7:44:00  F9 v1.0  B0005  CCAFS LC-40   
3              4  2012-10-08    0:35:00  F9 v1.0  B0006  CCAFS LC-40   
4              5  2013-03-01   15:10:00  F9 v1.0  B0007  CCAFS LC-40   

                                             Payload  Payload Mass (kg)  \
0               Dragon Spacecraft Qualification Unit                0.0   
1  Dragon demo flight C1, two CubeSats,  barrel o...                0.0   
2                             Dragon demo flight C2+              525.0   
3                                       SpaceX CRS-1              500.0   
4                                       SpaceX CRS-2              677.0   

       Orbit         Customer        Landing Outcome  class        Lat  \


In [14]:
# Step 6: Create a MarkerCluster object (for future visualization)
marker_cluster = MarkerCluster()

*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [15]:
# Step 7: Create a Folium Map and Add Markers for Each Launch
site_map = folium.Map(location=[28.5, -80.6], zoom_start=5)

for index, row in spacex_df.iterrows():
    # Add markers to the MarkerCluster
    folium.Marker(
        location=[row['Lat'], row['Long']],
        icon=folium.Icon(color=row['marker_color']),
        popup=folium.Popup(f"Launch Site: {row['Launch Site']}<br>Outcome: {'Success' if row['class'] == 1 else 'Failure'}")
    ).add_to(marker_cluster)

# Add MarkerCluster to the map
site_map.add_child(marker_cluster)

# Display the map
site_map

Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


## TASK 3: Calculate the distances between a launch site to its proximities


Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [20]:
import folium
from folium.plugins import MousePosition
from math import radians, sin, cos, sqrt, atan2

# Step 1: Define the function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    R = 6371  # Earth's radius in kilometers
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Apply Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    distance = R * c
    return distance

# Step 2: Add MousePosition to the map
site_map = folium.Map(location=[28.5, -80.6], zoom_start=5)
mouse_position = MousePosition(
    position='topright',
    separator=' | ',
    empty_string='',
    lng_first=True,
    num_digits=6,
    prefix="Coordinates:"
)

# Step 3: Mark down the coastline point and calculate the distance
# Example coordinates for the closest coastline point and a launch site
launch_site_lat = 28.562302
launch_site_lon = -80.577356
coastline_lat = 28.56367
coastline_lon = -80.57163

site_map.add_child(mouse_position)

Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


### Find coordinate of the closet coastline

In [21]:
# Calculate the distance between the launch site and coastline
distance_to_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)

### Display the distance between coastline point and launch site using the icon property 

### Create and add a folium.Marker on your selected closest coastline point on the map

In [22]:
# Initialize the map
site_map = folium.Map(location=[28.5, -80.6], zoom_start=10)

# Add a marker at the coastline point
distance_marker = folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(distance_to_coastline),
    )
)
site_map.add_child(distance_marker)

# Add a marker at the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    popup="Launch Site",
    icon=folium.Icon(color="green")
).add_to(site_map)

*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point

In [23]:
# Add a PolyLine between the launch site and coastline
coordinates = [[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]]
folium.PolyLine(locations=coordinates, weight=1, color='blue').add_to(site_map)

# Display the map
site_map

In [25]:
import folium
from folium.plugins import MousePosition
from math import radians, sin, cos, sqrt, atan2

# Function to calculate the distance between two coordinates
def calculate_distance(lat1, lon1, lat2, lon2):
    # Earth radius in KM
    R = 6371.0
    # Convert latitude and longitude to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    # Compute differences
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    # Haversine formula
    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Function to find an intermediate point at a specific distance
def find_intermediate_point(lat1, lon1, lat2, lon2, distance_km, total_distance_km):
    # Calculate the fraction of the total distance
    fraction = distance_km / total_distance_km
    # Compute intermediate latitude and longitude
    lat = lat1 + fraction * (lat2 - lat1)
    lon = lon1 + fraction * (lon2 - lon1)
    return lat, lon

# Coordinates for launch site and current coastline point
launch_site_lat = 28.563302
launch_site_lon = -80.576820
current_coastline_lat = 28.56367
current_coastline_lon = -80.57163

# Calculate total distance to the current coastline
total_distance = calculate_distance(launch_site_lat, launch_site_lon, current_coastline_lat, current_coastline_lon)

# Find the new point 0.90 KM away from the launch site
new_coastline_lat, new_coastline_lon = find_intermediate_point(
    launch_site_lat, launch_site_lon, current_coastline_lat, current_coastline_lon, 0.90, total_distance
)

# Create the map
site_map = folium.Map(location=[28.5, -80.6], zoom_start=15)

# Add a PolyLine to the map
coordinates = [[launch_site_lat, launch_site_lon], [new_coastline_lat, new_coastline_lon]]
folium.PolyLine(locations=coordinates, weight=2, color='blue').add_to(site_map)

# Add a marker for the new coastline point with the distance
distance_marker = folium.Marker(
    [new_coastline_lat, new_coastline_lon],
    icon=folium.DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html='<div style="font-size: 12; color:#d35400;"><b>{:10.2f} KM</b></div>'.format(0.90),
    )
)
site_map.add_child(distance_marker)

# Display the map
site_map

In [28]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Initialize the map
site_map = folium.Map(location=[28.563197, -80.576820], zoom_start=15)

# MarkerCluster setup for the launch sites
marker_cluster = MarkerCluster().add_to(site_map)

# Load data (ensure the spacex_df DataFrame is already loaded)
for index, record in spacex_df.iterrows():
    # Extract necessary information
    launch_site = record['Launch Site']
    lat = record['Lat']
    lon = record['Long']
    outcome = record['class']  # 1 for success, 0 for failure

    # Define marker color based on success or failure
    marker_color = 'green' if outcome == 1 else 'red'

    # Add the marker to the cluster
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color=marker_color, icon='ok' if outcome == 1 else 'remove'),
        popup=f"Site: {launch_site}<br>Outcome: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Add a blue line for the coastline distance
launch_site_lat, launch_site_lon = 28.563197, -80.576820  # Launch site coordinates
coastline_lat, coastline_lon = 28.564066, -80.567851  # Coastline coordinates

folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=1
).add_to(site_map)

# Add the distance as a marker
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color: #d35400;"><b>{distance:.2f} KM</b></div>',
    )
).add_to(site_map)

# Display the map
site_map

In [34]:
import folium
from folium.plugins import MarkerCluster
from folium.features import DivIcon

# Initialize the map centered at the primary launch site
site_map = folium.Map(location=[28.563197, -80.576820], zoom_start=15)

# MarkerCluster setup for launch outcomes
marker_cluster = MarkerCluster().add_to(site_map)

# Load data (ensure the spacex_df DataFrame is already loaded)
for index, record in spacex_df.iterrows():
    # Extract necessary information
    launch_site = record['Launch Site']
    lat = record['Lat']
    lon = record['Long']
    outcome = record['class']  # 1 for success, 0 for failure

    # Define marker color based on success or failure
    marker_color = 'green' if outcome == 1 else 'red'

    # Add the marker to the cluster
    folium.Marker(
        [lat, lon],
        icon=folium.Icon(color=marker_color, icon='ok' if outcome == 1 else 'remove'),
        popup=f"Site: {launch_site}<br>Outcome: {'Success' if outcome == 1 else 'Failure'}"
    ).add_to(marker_cluster)

# Add small red circle indicators on each launch site
launch_sites = spacex_df[['Launch Site', 'Lat', 'Long']].drop_duplicates()

for _, site in launch_sites.iterrows():
    folium.CircleMarker(
        location=[site['Lat'], site['Long']],
        radius=40,  # Small circle
        color='red',
        fill=True,
        fill_color='red',
        fill_opacity=0.5
    ).add_to(site_map)

# Add the blue line for the coastline distance
launch_site_lat, launch_site_lon = 28.563197, -80.576820  # Launch site coordinates
coastline_lat, coastline_lon = 28.564066, -80.567851  # Coastline coordinates

folium.PolyLine(
    locations=[[launch_site_lat, launch_site_lon], [coastline_lat, coastline_lon]],
    color='blue',
    weight=1
).add_to(site_map)

# Add the distance as a marker
distance = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
folium.Marker(
    [coastline_lat, coastline_lon],
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color: #d35400;"><b>{distance:.2f} KM</b></div>',
    )
).add_to(site_map)

# Display the map
site_map



Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


## Create a marker with distance to a closest city, railway, highway, etc.

## Draw a line between the marker to the launch site

In [35]:
import folium
from folium.plugins import MousePosition
from math import radians, cos, sin, sqrt, atan2
from folium.features import DivIcon

# Function to calculate distance between two points (Haversine formula)
def calculate_distance(lat1, lon1, lat2, lon2):
    R = 6371  # Earth radius in km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2) ** 2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2) ** 2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Initialize the map centered around a specific launch site
launch_site_lat = 28.563197
launch_site_lon = -80.576820
site_map = folium.Map(location=[launch_site_lat, launch_site_lon], zoom_start=15)

# Add MousePosition plugin to get coordinates
mouse_position = MousePosition(
    position="topright",
    separator=" | ",
    prefix="Coordinates:",
    num_digits=6
)
site_map.add_child(mouse_position)

# Coordinates for closest city, railway, and highway (example coordinates)
city_lat, city_lon = 28.612833, -80.807046  # Melbourne, FL (example)
railway_lat, railway_lon = 28.5721, -80.5853  # Closest railway point (example)
highway_lat, highway_lon = 28.5626, -80.5776  # Closest highway point (example)

# Add markers for the locations with distances
locations = {
    "City": (city_lat, city_lon),
    "Railway": (railway_lat, railway_lon),
    "Highway": (highway_lat, highway_lon),
}

for name, (lat, lon) in locations.items():
    distance = calculate_distance(launch_site_lat, launch_site_lon, lat, lon)
    # Add a marker for the location
    folium.Marker(
        [lat, lon],
        icon=DivIcon(
            icon_size=(20, 20),
            icon_anchor=(0, 0),
            html=f'<div style="font-size: 12px; color: #d35400;"><b>{distance:.2f} KM</b></div>',
        ),
        popup=f"{name}: {distance:.2f} KM"
    ).add_to(site_map)
    # Draw a line between the launch site and the location
    folium.PolyLine(
        locations=[[launch_site_lat, launch_site_lon], [lat, lon]],
        color="blue",
        weight=2
    ).add_to(site_map)

# Add a marker for the launch site
folium.Marker(
    [launch_site_lat, launch_site_lon],
    icon=folium.Icon(color="green", icon="rocket"),
    popup="Launch Site"
).add_to(site_map)

# Display the map
site_map




## Optional Questions:

After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.
